Après avoir exploré le site DATAtourisme, Nous avons listé 4 façons d'accéder aux données Datatourisme:


1. Création d'un "Flux" quotidien, mettant à disposition chaque jour un répertoire zippé de fichiers JSON, à télécharger
2. Création d'un "Flux" quotidien, mettant à disposition, chaque jour, un fichier CSV à télécharger
3. Exports simplifiés : Téléchargement immédiat d'un fichier CSV à partir de la base de données de ADN Tourisme hébergée sur le site datagouv.fr. Nous avons repéré 25 type de fichiers dont les mises à jour sont uploadées régulièrement. Nous les avons répértorié dans le google sheets ci-après : 
https://docs.google.com/spreadsheets/d/1kB-IQgFPt5P-6JUSPj-u6oht8sFrpMLWFd8kztJrvCc/edit?usp=sharing
4. Utilisation de l'API Datatourisme par l'intermédiaire d'une requète curl


La zone géographique choisie pour le projet est le département de La Réunion.


A ce stade nous avons privilégié les données issues de 3. pour les raisons suivantes:


- 2. produit un fichier CSV qui est moins complet que 3. et plus difficile à exploiter directement
- 1. produit une collection de fichiers JSON (un par POI), qui contient des informations supplémentaires par rapport aux CSV (thème des POI, accès handicapé, horaires d'ouverture pour certaines zones géographiques...), mais que nous avons jugées non pertinentes pour le projet dans le cas de La Réunion (en particulier les horaires d'ouverture ne sont pas présents)
- 4. serait la méthode à privilégier (sous réserve que les données accessibles par cette méthode soient complètes) afin d'automatiser l'actualisation du projet. Nous aborderons cette méthode en fin de projet au cours de l'étape d'automatisation (si nous avons le temps de l'aborder)


Avant d'importer le fichier csv, il faut vérifier s'il y a des données corrompues. On peut trouver les lignes qui ne contiennent pas l'URL avec l'identifiant du POI avec la commande suivante :


grep -P -v -n -A1 "/[a-z0-9]{8}\-[a-z0-9]{4}\-[a-z0-9]{4}\-[a-z0-9]{4}\-[a-z0-9]{12}" datatourisme-reg-reu-20230118.csv


Dans le fichier csv de la Réunion, il y avait 4 entrées qui étaient réparties sur plusieurs lignes. En supprimant les retours à la ligne non voulus, les données ont été corrigées et ne seront pas perdues. Les corrections sont faites dans le code ci-dessous.


Dans la suite de ce notebook, nous utilisons la librairie Pandas de Python afin de créer un DataFrame à partir du fichier csv, ce qui nous a permis de visualiser, nettoyer, filtrer et faire une rapide etude statistique des données issues de 3.

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
import sys
import re

# ce script corrige les entrées réparties sur plusieurs lignes en les remettant sur une seule ligne
# en supprimant le retour à la ligne (et l'espace précédent s'il y en a un), en ajoutant une virgule

# usage: python3 fix-split-lines.py INPUT OUTPUT ## si le script est utilisé hors colab

# fichier INPUT: .csv téléchargé depuis
# https://www.data.gouv.fr/fr/datasets/datatourisme-la-base-nationale-des-donnees-publiques-dinformation-touristique-en-open-data/

#inFile = sys.argv[1]  ## si le script est utilisé hors colab
#outFile = sys.argv[2] ## si le script est utilisé hors colab

inFile = 'datatourisme-reg-reu-20230118.csv'
outFile = 'datatourisme-reg-reu-20230118_fix.csv'

header_re = re.compile(r'Nom_du_POI')
uri_re = re.compile(r'/[a-z0-9]{8}-[a-z0-9]{4}-[a-z0-9]{4}-[a-z0-9]{4}-[a-z0-9]{12}\n')

#with open (inFile, encoding='utf-8') as f_in, open(outFile, 'w', encoding='utf-8') as f_out: ## si le script est utilisé hors colab
with open ('/content/drive/MyDrive/datatourisme-reg-reu-20230111.csv', encoding='utf-8') as f_in, open('/content/drive/MyDrive/datatourisme-reg-reu_fix.csv', 'w', encoding='utf-8') as f_out:
#with open ('/content/datatourisme-reg-reu-20230118.csv', encoding='utf-8') as f_in, open('/content/datatourisme-reg-reu-20230118_fix.csv', 'w', encoding='utf-8') as f_out: ## selon où le fichier se trouve
	for line in f_in:
		if header_re.search(line) or uri_re.search(line):
			f_out.write(line) # header et lignes correctes restent inchangés
		else:
			f_out.write(line.rstrip() + ', ') # lignes incomplètes : on supprime (l'espace et) le retour à la ligne

In [32]:
#création d'un dataframe à partir du fichier csv et visualisation
import pandas as pd
df = pd.read_csv(r'/content/drive/MyDrive/datatourisme-reg-reu_fix.csv',sep=',',header=0)

df.head()

,Nom_du_POI,Categories_de_POI,Latitude,Longitude,Adresse_postale,Code_postal_et_commune,Periodes_regroupees,Covid19_mesures_specifiques,Covid19_est_en_activite,Covid19_periodes_d_ouvertures_confirmees,Createur_de_la_donnee,SIT_diffuseur,Date_de_mise_a_jour,Contacts_du_POI,Classements_du_POI,Description,URI_ID_du_POI
0,Gîtes de Boucan Canot n° 58 (Les),https://www.datatourisme.fr/ontology/core#Plac...,-21.034302,55.230223,"255, chemin de la Vanille - Boucan Canot",97434#Saint-Paul,NaN,NaN,NaN,NaN,Soubik,ADT Réunion,2020-01-23,Gîtes de Boucan Canot n° 58 (Les)#+33 6 92 65 ...,4 étoiles#Classement officiel des hébergements...,NaN,https://data.datatourisme.fr/54/27441221-b465-...
1,Gîtes de Boucan Canot n°141 (Les),https://www.datatourisme.fr/ontology/core#Plac...,-21.034228,55.230653,"255, chemin de la Vanille - Boucan Canot",97434#Saint-Paul,NaN,NaN,NaN,NaN,Soubik,ADT Réunion,2020-01-23,Gîtes de Boucan Canot n°141 (Les)#+33 6 92 65 ...,4 épis / Premium#Gîtes de France,NaN,https://data.datatourisme.fr/54/ce516e2d-f9f2-...
2,Aux Nids d'Hirondelles,https://www.datatourisme.fr/ontology/core#Plac...,-21.343197,55.479924,"11 bis, rue Amiral Lacaze - Terre Sainte",97410#Saint-Pierre,NaN,NaN,NaN,NaN,Soubik,ADT Réunion,2020-04-02,Aux Nids d'Hirondelles#+33 2 62 42 17 23#d_mat...,2 étoiles#Classement officiel des hébergements...,NaN,https://data.datatourisme.fr/54/3125c269-e57c-...
3,Capucine les Hauts,https://www.datatourisme.fr/ontology/core#Plac...,-21.260916,55.431705,"27, rue Calmette et Guérin - Ruisseau",97421#Saint-Louis,NaN,NaN,NaN,NaN,Soubik,ADT Réunion,2022-08-24,Capucine les Hauts#+33 2 62 91 36 59#loc.rosa@...,3 épis / Confort#Gîtes de France,NaN,https://data.datatourisme.fr/54/318cd437-537a-...
4,Tikazbourbon,https://www.datatourisme.fr/ontology/core#Plac...,-21.300900,55.501714,"105 A, ancienne RN 3 - Condé 400",97410#Saint-Pierre,NaN,NaN,NaN,NaN,Soubik,ADT Réunion,2020-03-20,Tikazbourbon#+33 2 62 22 63 99#tkb974@gmail.co...,2 étoiles#Classement officiel des hébergements...,NaN,https://data.datatourisme.fr/54/32e09e05-74be-...


In [33]:
#affichage des dimensions du Dataframe
df.shape


(3571, 17)

In [34]:
#extraction d'un url de la colonne 'URI_ID_du_POI''
df.loc[9,'URI_ID_du_POI']

'https://data.datatourisme.fr/54/669654df-f23e-3af2-bb63-10765b9325a2'

In [35]:
#creation d'une colonne avec le numéro de la région 
df['region']=df['URI_ID_du_POI'].apply(lambda x: x.rsplit('/',2)[1])
#creation d'une colonne avec ID (à utiliser comme index et clé primaire sur sql)
df['ID']=df['URI_ID_du_POI'].apply(lambda x: x.rsplit('/',2)[2])

In [36]:
#split de la colonne code_postal_et_commune en 2 colonnes code_postal / commune
df['Code_postal_et_commune']=df['Code_postal_et_commune'].apply(lambda x: x.split('#'))

df['commune']=df['Code_postal_et_commune'].apply(lambda x:x[1])

df['code_postal']=df['Code_postal_et_commune'].apply(lambda x:x[0])

In [37]:
#extraction d'un url de la colonne 'catégories_de_poi' pour accès à la page et analyse
mode=df['Categories_de_POI'].mode()
print(mode[0])

https://www.datatourisme.fr/ontology/core#PlaceOfInterest|https://www.datatourisme.fr/ontology/core#PointOfInterest|https://www.datatourisme.fr/ontology/core#Accommodation|https://www.datatourisme.fr/ontology/core#RentalAccommodation|http://schema.org/Accommodation|http://schema.org/LodgingBusiness


In [38]:
#extraction des catégories de POI de l'url
def splitter(row):
  l=row.split('|')
  m=[i.split('#') for i in l]
  cats = [j[1] for j in m if len(j)==2]
  return cats

df['categories'] = df['Categories_de_POI'].apply(lambda row: splitter(row))

In [39]:
# filtrage des catégories pour ne garder que la plus petite

import json
with open ('/content/drive/MyDrive/ontology_hierarchy_ascending_list.txt', encoding='utf-8') as f_in:
  cat_chain_list = json.load(f_in)
from collections import defaultdict
is_higher_than = defaultdict(set)
all_categories = set()
for l in cat_chain_list:
  if "PlaceOfInterest" in l: l.remove("PlaceOfInterest")
  if "PointOfInterest" in l: l.remove("PointOfInterest")
  for i in range(0, len(l)):
    all_categories.add(l[i])
    for j in range(i + 1, len(l)):
      is_higher_than[l[j]].add(l[i])

excluded_categories = ['FitnessCenter']

def filter_and_keep_lowest_cat(categories):
  lower_categories = []
  for i in range(0, len(categories)):
    is_lower_category = True
    if categories[i] not in all_categories or categories[i] in excluded_categories:
      continue
    for j in range(0, len(categories)):
      if i != j and categories[j] in all_categories:
        if categories[j] in is_higher_than.get(categories[i], []):
          is_lower_category = False
    if is_lower_category:
      lower_categories.append(categories[i])
  return lower_categories

df['categories']=df['categories'].apply(lambda categories: filter_and_keep_lowest_cat(categories))


In [40]:
# ajouter la catégorie la plus générale (hors PlaceOfInterest et PointOfInterest) dans une nouvelle colonne

with open ('/content/drive/MyDrive/ontology_hierarchy_descending_list.txt', encoding='utf-8') as f_in:
  cat_chain_list = json.load(f_in)
def add_global_cat(categories):
  lowest_cats = categories
  highest_cats = []
  for cat in lowest_cats:
    for l in cat_chain_list:
      if cat in l:
        highest_cats.append(l[2])
        break;
  return highest_cats
  print(highest_cats)

df['supercategories']=df['categories'].apply(lambda categories: add_global_cat(categories))

#df.to_csv('/content/drive/MyDrive/reunion_check_supercategories_tabs.csv', sep='\t')

In [41]:
#supression de catégories PointOfInterest and PlaceOfInterest: #déjà fait dans le filtrage des catégories les plus fines
# df['categories']=df['categories'].apply(lambda list: [i for i in list if i not in ['PointOfInterest','PlaceOfInterest']])

#vérification du nombre maximal de niveaux de catégories
max_cats = max(len(i) for i in df['categories'])

#split des catégories (list) en plusieurs colonnes
categories = pd.DataFrame(df['categories'].to_list(), columns=['subclass_'+str(i) for i in range(max_cats)])

#split des catégories (list) en plusieurs colonnes
supercategories = pd.DataFrame(df['supercategories'].to_list(), columns=['superclass_'+str(i) for i in range(max_cats)])

#ajout des colonnes catégories et supercategories à df
df = pd.concat([df,categories,supercategories],axis=1)

# suppression des lignes où il n'y a pas de catégorie
df = df[~(df['categories'].apply(len) == 0)]

#df.to_csv('/content/drive/MyDrive/reunion_check_subclass_superclass_tabs.csv', sep='\t')

In [42]:
#exploration des différentes catégories par niveau

subclass_cols = ['subclass_0','subclass_1']
subclass_counts = df[subclass_cols].stack().value_counts()
print("subclass counts:")
print(subclass_counts)

superclass_cols = ['superclass_0','superclass_1']
superclass_counts = df[superclass_cols].stack().value_counts()
print("superclass counts:")
print(superclass_counts)

subclass counts:
Restaurant                       883
FoodEstablishment                317
LeisureSportActivityProvider     212
NaturalHeritage                  121
CraftsmanShop                    118
CulturalSite                      85
NauticalCentre                    70
LocalProductsShop                 52
TechnicalHeritage                 35
FarmhouseInn                      27
PointOfView                       13
ArtGalleryOrExhibitionGallery     11
Pond                              11
Source                            11
Beach                             11
Waterfall                          9
ThemePark                          8
MooringArea                        8
Mountain                           7
Church                             5
Forest                             5
Coastline                          3
Temple                             3
Col                                2
Volcano                            1
Bridge                             1
Mosque               

In [43]:
#suppression de colonnes superflus ou vides
df=df.drop(columns = ['categories','supercategories','Code_postal_et_commune','Covid19_mesures_specifiques','Covid19_est_en_activite','Covid19_periodes_d_ouvertures_confirmees','Categories_de_POI','Createur_de_la_donnee','SIT_diffuseur','Classements_du_POI','URI_ID_du_POI'])

df=df.dropna(axis=1,how='all')

df


,Nom_du_POI,Latitude,Longitude,Adresse_postale,Date_de_mise_a_jour,Contacts_du_POI,Description,region,ID,commune,code_postal,subclass_0,subclass_1,superclass_0,superclass_1
12,Lentille des Gourmets (La),-21.135884,55.479501,"40 E, chemin des Trois Mares",2022-05-30,Lentille des Gourmets (La)#+33 2 62 31 85 85#h...,NaN,54,187a27ff-8ad2-3298-9b2f-064023dd042b,Cilaos,97413,Restaurant,None,FoodEstablishment,None
20,Neptune (Le),-21.341440,55.462664,"117, boulevard Hubert Delisle",2021-07-13,Neptune (Le)#+33 2 62 61 61 61#reservation@leb...,NaN,54,ac5515a8-d78e-35ed-a51e-6fc6b1ddbb2b,Saint-Pierre,97410,LeisureSportActivityProvider,None,ActivityProvider,None
21,Grand Large (Le),-21.341440,55.462664,"117, boulevard Hubert Delisle",2021-07-13,Grand Large (Le)#+33 2 62 61 61 61#reservation...,NaN,54,43079596-c868-3cfe-a280-aeaf15d5666c,Saint-Pierre,97410,Restaurant,None,FoodEstablishment,None
29,Concorde (Le),-20.896258,55.447236,"91 bis, allée des Topazes - Bellepierre",2019-10-10,Concorde (Le)#+33 2 62 51 51 51#info@hotel-bel...,NaN,54,f2a140d5-152c-300b-accb-67619012a89a,Saint-Denis,97400,Restaurant,LeisureSportActivityProvider,FoodEstablishment,ActivityProvider
34,Jardin de la Maison d'Edith (Le),-20.889919,55.407147,"59, chemin Commins",2020-05-07,Jardin de la Maison d'Edith (Le)#+33 6 92 69 6...,NaN,54,9c77eb5d-8fd9-3700-8e77-e830870e9669,Saint-Denis,97417,CulturalSite,None,CulturalSite,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3563,Mosaïk Kitchen,-21.336766,55.479371,"10, rue du Presbytère",2019-10-10,Mosaïk Kitchen#+33 2 62 24 81 37#mozaikkitchen...,NaN,54,debb6715-f9ac-336d-9de2-c7e8dca78ad8,Saint-Pierre,97410,Restaurant,None,FoodEstablishment,None
3564,Restaurant Ti Zardin,-21.260194,55.357715,195 avenue Raymond Barre,2022-07-05,Restaurant Ti Zardin#+33 2 62 39 51 21##,NaN,54,e08499a1-f0f2-356f-9459-655b865aa9c2,L'Étang-Salé,97427,Restaurant,None,FoodEstablishment,None
3566,Restaurant Le Métis,-21.159397,55.823763,378 bis Route Nationale 2,2021-02-08,Restaurant Le Métis#+33 6 92 53 77 33#fabienne...,NaN,54,e0f6d90d-333d-35f4-b2a9-a413865e6ce7,Sainte-Rose,97439,Restaurant,None,FoodEstablishment,None
3567,Ô Tipikemen Kréol,-21.061952,55.520576,10 impasse des Aubépines,2022-01-11,Ô Tipikemen Kréol#+33 6 93 30 30 74##,NaN,54,e0fd47b0-c629-3058-a838-5e10a53550de,Salazie,97433,Restaurant,None,FoodEstablishment,None


In [44]:
# delete duplicates
df=df.drop_duplicates(subset=['Nom_du_POI','Latitude','Longitude'])

In [45]:
#définir l'ID comme index du df
df_clean=df.set_index('ID',drop=True)

In [46]:
#creation d'un df avec uniquement les coordonnées de chaque poi (pour sql)
df_coordonnees=df_clean[['Nom_du_POI','Latitude','Longitude']]
df_coordonnees

,Nom_du_POI,Latitude,Longitude
ID,,,
187a27ff-8ad2-3298-9b2f-064023dd042b,Lentille des Gourmets (La),-21.135884,55.479501
ac5515a8-d78e-35ed-a51e-6fc6b1ddbb2b,Neptune (Le),-21.341440,55.462664
43079596-c868-3cfe-a280-aeaf15d5666c,Grand Large (Le),-21.341440,55.462664
f2a140d5-152c-300b-accb-67619012a89a,Concorde (Le),-20.896258,55.447236
9c77eb5d-8fd9-3700-8e77-e830870e9669,Jardin de la Maison d'Edith (Le),-20.889919,55.407147
...,...,...,...
debb6715-f9ac-336d-9de2-c7e8dca78ad8,Mosaïk Kitchen,-21.336766,55.479371
e08499a1-f0f2-356f-9459-655b865aa9c2,Restaurant Ti Zardin,-21.260194,55.357715
e0f6d90d-333d-35f4-b2a9-a413865e6ce7,Restaurant Le Métis,-21.159397,55.823763


In [47]:
#creation d'un dataframe de correspondance entre code postal et commune (pour sql)
df_ville_cp=df_clean[['commune','code_postal','region']]
df_ville_cp.drop_duplicates('code_postal',inplace=True)
df_ville_cp=df_ville_cp.sort_values(by='commune')
df_ville_cp_index=df_ville_cp.set_index('code_postal', drop = True)
df_ville_cp_index

<ipython-input-47-53db21f87a92>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ville_cp.drop_duplicates('code_postal',inplace=True)


,commune,region
code_postal,,
97412,Bras-Panon,54
97413,Cilaos,54
97414,Entre-Deux,54
97427,L'Étang-Salé,54
97431,La Plaine-des-Palmistes,54
97419,La Possession,54
97420,Le Port,54
97430,Le Tampon,54
97418,Le Tampon,54


In [48]:
#creation d'un tableau Food uniquement pour création d'un eventuel features de proposition de restaurants pour chaque itineraire
df_food=df_clean[(df_clean['superclass_0'] == 'FoodEstablishment') | (df_clean['superclass_1']=='FoodEstablishment')]
df_food

,Nom_du_POI,Latitude,Longitude,Adresse_postale,Date_de_mise_a_jour,Contacts_du_POI,Description,region,commune,code_postal,subclass_0,subclass_1,superclass_0,superclass_1
ID,,,,,,,,,,,,,,
187a27ff-8ad2-3298-9b2f-064023dd042b,Lentille des Gourmets (La),-21.135884,55.479501,"40 E, chemin des Trois Mares",2022-05-30,Lentille des Gourmets (La)#+33 2 62 31 85 85#h...,NaN,54,Cilaos,97413,Restaurant,None,FoodEstablishment,None
43079596-c868-3cfe-a280-aeaf15d5666c,Grand Large (Le),-21.341440,55.462664,"117, boulevard Hubert Delisle",2021-07-13,Grand Large (Le)#+33 2 62 61 61 61#reservation...,NaN,54,Saint-Pierre,97410,Restaurant,None,FoodEstablishment,None
f2a140d5-152c-300b-accb-67619012a89a,Concorde (Le),-20.896258,55.447236,"91 bis, allée des Topazes - Bellepierre",2019-10-10,Concorde (Le)#+33 2 62 51 51 51#info@hotel-bel...,NaN,54,Saint-Denis,97400,Restaurant,LeisureSportActivityProvider,FoodEstablishment,ActivityProvider
fc712c6c-ff99-38a9-b2cb-f6f098200184,Baies Roses,-20.949981,55.341023,"142, chemin Bœuf Mort",2020-02-10,Baies Roses#+33 2 62 44 66 88##https://www.lod...,NaN,54,La Possession,97419,Restaurant,None,FoodEstablishment,None
1dbd44fe-adb1-30c5-9b0c-a80c877df6b8,Oasis (L'),-20.873973,55.447235,"2, rue Doret",2019-10-10,Oasis (L')#+33 2 62 21 80 20##,NaN,54,Saint-Denis,97400,Restaurant,None,FoodEstablishment,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
debb6715-f9ac-336d-9de2-c7e8dca78ad8,Mosaïk Kitchen,-21.336766,55.479371,"10, rue du Presbytère",2019-10-10,Mosaïk Kitchen#+33 2 62 24 81 37#mozaikkitchen...,NaN,54,Saint-Pierre,97410,Restaurant,None,FoodEstablishment,None
e08499a1-f0f2-356f-9459-655b865aa9c2,Restaurant Ti Zardin,-21.260194,55.357715,195 avenue Raymond Barre,2022-07-05,Restaurant Ti Zardin#+33 2 62 39 51 21##,NaN,54,L'Étang-Salé,97427,Restaurant,None,FoodEstablishment,None
e0f6d90d-333d-35f4-b2a9-a413865e6ce7,Restaurant Le Métis,-21.159397,55.823763,378 bis Route Nationale 2,2021-02-08,Restaurant Le Métis#+33 6 92 53 77 33#fabienne...,NaN,54,Sainte-Rose,97439,Restaurant,None,FoodEstablishment,None


In [49]:
#creation d'un tableau Store uniquement pour création d'un eventuel features de proposition de magasins de produits locaux
df_store=df_clean[(df_clean['superclass_0'] == 'Store') | (df_clean['superclass_1']=='Store')]
df_store

,Nom_du_POI,Latitude,Longitude,Adresse_postale,Date_de_mise_a_jour,Contacts_du_POI,Description,region,commune,code_postal,subclass_0,subclass_1,superclass_0,superclass_1
ID,,,,,,,,,,,,,,
e99a3708-7e99-34f3-9b1d-5218c73ec817,Passion Outremer,-20.891495,55.512258,Aéroport Roland Garros,2019-10-10,Passion Outremer#+33 6 92 95 21 58#info@passio...,NaN,54,Sainte-Marie,97438,LocalProductsShop,None,Store,None
3183380a-472e-3387-8e62-d7ba91636e58,Essences et Couleurs,-21.277511,55.467450,"RD 26, pont du Bras de la Plaine - Kiosque du ...",2019-10-10,Essences et Couleurs#+33 6 93 93 34 81#celine....,NaN,54,Saint-Pierre,97432,CraftsmanShop,None,Store,None
d7734add-e6f2-37eb-971c-e810ffedf198,Autre Façon,-21.050979,55.224360,"57, rue du Général de Gaulle",2019-10-10,Autre Façon#+33 2 62 24 05 03##,NaN,54,Saint-Paul,97434,CraftsmanShop,None,Store,None
d801712a-20f8-3f43-99e2-f65ea634ef1e,Souvenirs Lontan - Chez Guylène,-21.007887,55.272493,"59, rue Marius & Ary Leblond - Marché Couvert",2019-10-10,Souvenirs Lontan - Chez Guylène#+33 6 92 69 21...,NaN,54,Saint-Paul,97460,CraftsmanShop,None,Store,None
6d77df79-01b5-3bc0-ae6c-402f1c41246e,Souvenirs Créoles,-21.065596,55.519291,"51 ter, rue du Général de Gaulle - Hell-Bourg",2022-01-24,Souvenirs Créoles#+33 2 62 31 02 05##,NaN,54,Salazie,97433,CraftsmanShop,None,Store,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cdf0259f-d2fa-3744-b6f7-d82395116d7d,Tortue fait Maison (La),-21.337891,55.471640,68 rue Caumont,2021-02-03,Tortue fait Maison (La)#+33 6 63 10 19 56#bonj...,NaN,54,Saint-Pierre,97410,CraftsmanShop,None,Store,None
d3a74eda-903f-3fc6-8c93-7a64795b7cba,Mozaïk Bambou,-21.204801,55.582398,"71, chemin Alfred Picard - Bourg Murat",2019-10-10,Mozaïk Bambou#+33 2 62 59 22 06#decorboismosai...,NaN,54,Le Tampon,97418,CraftsmanShop,None,Store,None
d7d4482c-15ac-3361-8396-1bd9f622859c,Moana Créations,-21.335962,55.576119,"8, chemin Léopold Lebon",2019-10-10,Moana Créations#+33 6 92 42 17 61#moanacreatio...,NaN,54,Petite-Île,97429,CraftsmanShop,None,Store,None


In [50]:
#creation d'un df avec uniquement les poi (exclude food)
df_poi = df_clean[(~df_clean[['superclass_0','superclass_1']].isin(['FoodEstablishment'])).all(1)]

In [51]:
#creation d'un df avec air activity (parapente, ULM etc)
df_airactivity = df_poi[((df_poi['subclass_0'] == 'LeisureSportActivityProvider')|(df_poi['subclass_1'] == 'LeisureSportActivityProvider')) & ((df_poi['Nom_du_POI'].str.contains('Parapente'))|(df_poi['Nom_du_POI'].str.contains('Air'))|(df_poi['Nom_du_POI'].str.contains('Aéro'))|(df_poi['Nom_du_POI'].str.contains('Paramoteur'))|(df_poi['Nom_du_POI'].str.contains('Ulm'))|(df_poi['Nom_du_POI'].str.contains('ULM'))|(df_poi['Nom_du_POI'].str.contains('Héli'))|(df_poi['Nom_du_POI'].str.contains('Aile'))|(df_poi['Adresse_postale'].str.contains('ULM')))]

In [52]:
#creation d'un df avec uniquement les activity provider
#df_activity=df_poi[(df_poi['category_0'] == 'ActivityProvider') | (df_poi['category_0'] == 'SportsAndLeisurePlace')] 
#df_activity=df_poi[(df_poi['category_0'] == 'LeisureSportActivityProvider') | (df_poi['category_0'] == 'SportsAndLeisurePlace')] 


In [53]:
#creation d'un df outdoor (air activity, nautical centre, natural heritage (subclass))
# keep waterfall, mountain, beach etc as POI; classify NaturalHeritage without subclass as activity
df_nauticalcentre = df_poi[(df_poi['subclass_0'] == 'NauticalCentre')|(df_poi['subclass_1'] == 'NauticalCentre')]
df_subnaturalheritage = df_poi[(df_poi['subclass_0'] == 'NaturalHeritage')|(df_poi['subclass_1'] == 'NaturalHeritage')]
df_outdooractivity = df_airactivity.append([df_nauticalcentre,df_subnaturalheritage])
df_outdooractivity.info()

subclass_cols = ['subclass_0','subclass_1']
subclass_counts = df_outdooractivity[subclass_cols].stack().value_counts()
print("subclass counts:")
print(subclass_counts)

superclass_cols = ['superclass_0','superclass_1']
superclass_counts = df_outdooractivity[superclass_cols].stack().value_counts()
print("superclass counts:")
print(superclass_counts)

<class 'pandas.core.frame.DataFrame'>
Index: 234 entries, 8edffc14-1e48-3ce6-af3a-e2fd6816fb0d to cf065482-6a87-3870-a70d-23edd74598a4
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Nom_du_POI           234 non-null    object 
 1   Latitude             234 non-null    float64
 2   Longitude            234 non-null    float64
 3   Adresse_postale      234 non-null    object 
 4   Date_de_mise_a_jour  234 non-null    object 
 5   Contacts_du_POI      234 non-null    object 
 6   Description          0 non-null      object 
 7   region               234 non-null    object 
 8   commune              234 non-null    object 
 9   code_postal          234 non-null    object 
 10  subclass_0           234 non-null    object 
 11  subclass_1           2 non-null      object 
 12  superclass_0         234 non-null    object 
 13  superclass_1         2 non-null      object 
dtypes: float64(2), object(12)
m

<ipython-input-53-4db654f9ac18>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_outdooractivity = df_airactivity.append([df_nauticalcentre,df_subnaturalheritage])


In [54]:
#creation d'un df avec uniquement les poi (exclude food, store and df outdoor (air activity, nautical centre, natural heritage (subclass)))
df_poi_2=pd.concat([df_poi, df_outdooractivity]).drop_duplicates(keep=False)
df_poi_2 = df_poi_2[(~df_poi_2[['superclass_0','superclass_1']].isin(['ActivityProvider','FoodEstablishment','SportsAndLeisurePlace','Store'])).all(1)]
df_poi_2.info()

subclass_cols = ['subclass_0','subclass_1']
subclass_counts = df_poi_2[subclass_cols].stack().value_counts()
print("subclass counts:")
print(subclass_counts)

superclass_cols = ['superclass_0','superclass_1']
superclass_counts = df_poi_2[superclass_cols].stack().value_counts()
print("superclass counts:")
print(superclass_counts)

<class 'pandas.core.frame.DataFrame'>
Index: 189 entries, 9c77eb5d-8fd9-3700-8e77-e830870e9669 to 5e11df20-9677-3365-b258-84cc1caa3edd
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Nom_du_POI           189 non-null    object 
 1   Latitude             189 non-null    float64
 2   Longitude            189 non-null    float64
 3   Adresse_postale      103 non-null    object 
 4   Date_de_mise_a_jour  189 non-null    object 
 5   Contacts_du_POI      103 non-null    object 
 6   Description          28 non-null     object 
 7   region               189 non-null    object 
 8   commune              189 non-null    object 
 9   code_postal          189 non-null    object 
 10  subclass_0           189 non-null    object 
 11  subclass_1           11 non-null     object 
 12  superclass_0         189 non-null    object 
 13  superclass_1         11 non-null     object 
dtypes: float64(2), object(12)
m

In [55]:
#repérage des poi incontournables
#keywords = ["Piton des Neiges",'Piton de la Fournaise','Voile de la Mariée','Hermitage', 'Plaine des Sables','Trou de Fer','Belvédère du Maïdo',"Gouffre de l'Etang-Salé",'Plage de Boucan Canot']
keywords = ["Piton des Neiges",'Piton de la Fournaise','Voile de la Mariée','Plage de l’Hermitage, La Passe', 'Plaine des Sables','Trou de Fer','Belvédère du Maïdo',"Gouffre de l'Etang-Salé",'Plage de Boucan Canot','Maison Folio']
df_best = df_poi[df_clean["Nom_du_POI"].apply(lambda x: any(k in x for k in keywords))]
df_best = df_best[-(df_best['Nom_du_POI'] == 'Salle Multimédia Piton des Neiges')]
df_best
df_best = df_poi[df_clean["Nom_du_POI"].apply(lambda x: any(k in x for k in keywords))]
df_best = df_best[-(df_best['Nom_du_POI'] == 'Salle Multimédia Piton des Neiges')]
df_best

<ipython-input-55-83da517f46d7>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_best = df_poi[df_clean["Nom_du_POI"].apply(lambda x: any(k in x for k in keywords))]
<ipython-input-55-83da517f46d7>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_best = df_poi[df_clean["Nom_du_POI"].apply(lambda x: any(k in x for k in keywords))]


,Nom_du_POI,Latitude,Longitude,Adresse_postale,Date_de_mise_a_jour,Contacts_du_POI,Description,region,commune,code_postal,subclass_0,subclass_1,superclass_0,superclass_1
ID,,,,,,,,,,,,,,
851e3d68-ca94-3193-98b7-052a04e17740,Maison Folio,-21.065045,55.518178,"20, rue Amiral Lacaze",2022-12-14,Maison Folio#+33 2 62 47 80 98#foliomaison@gma...,NaN,54,Salazie,97433,CulturalSite,None,CulturalSite,None
b1237276-bb2e-3d84-8974-eb49f9f5da11,Piton de la Fournaise,-21.243400,55.714733,NaN,2023-01-08,NaN,Cratère Dolomieu,54,Sainte-Rose,97439,Volcano,None,NaturalHeritage,None
76b40289-8ed2-3a14-b2a0-41c486054536,Plaine des Sables,-21.231197,55.649654,NaN,2023-01-08,NaN,Un arrêt s’impose pour admirer la Plaine des S...,54,Sainte-Rose,97439,PointOfView,None,NaturalHeritage,None
793d6f36-0e6d-30e9-9c7a-92c58a7a753a,Piton des Neiges,-21.097733,55.479947,NaN,2023-01-08,NaN,Le plus haut sommet de l'Océan indien,54,Cilaos,97413,Mountain,None,NaturalHeritage,None
86b7407d-9dd2-3628-9509-10d440a93a19,Trou de Fer,-21.037804,55.555272,NaN,2023-01-08,NaN,NaN,54,Salazie,97433,PointOfView,None,NaturalHeritage,None
646e3e7e-f9de-3802-94fa-1a1e4b582ea4,Voile de la Mariée,-21.039401,55.536248,NaN,2023-01-08,NaN,NaN,54,Salazie,97433,Waterfall,None,NaturalHeritage,None
854b2f3e-6650-3562-b9f3-36869fbb1937,Gouffre de l'Etang-Salé,-21.280060,55.340999,NaN,2023-01-08,NaN,NaN,54,L'Étang-Salé,97427,Coastline,None,NaturalHeritage,None
f4ccedb1-2fbf-31c4-ae68-cbf28ba930c9,Belvédère du Maïdo,-21.070640,55.387862,NaN,2023-01-08,NaN,Un point de vue exceptionnel sur le cirque de ...,54,Saint-Paul,97423,PointOfView,None,NaturalHeritage,None
fb1bd224-b864-381a-83e1-a787a950511d,Plage de Boucan Canot,-21.027624,55.225991,NaN,2023-01-08,NaN,"Sable blanc, profondeur et coraux",54,Saint-Paul,97434,Beach,None,NaturalHeritage,None


In [56]:
df_poi_3 = pd.merge(df_poi_2,df_best, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
         

In [57]:
print('les dimensions du df_clean global sont: ',df_clean.shape)
print('les dimensions du df_food sont:',df_food.shape)
#print('les dimensions du df_activity sont:',df_activity.shape)
print('les dimensions du df_activity sont:',df_outdooractivity.shape)
print('les dimensions du df_poi avec ActivityProvider sont:',df_poi.shape)
print('les dimensions du df_poi_2 sans ActivityProvider sont:',df_poi_2.shape)

les dimensions du df_clean global sont:  (1952, 14)
les dimensions du df_food sont: (1226, 14)
les dimensions du df_activity sont: (234, 14)
les dimensions du df_poi avec ActivityProvider sont: (726, 14)
les dimensions du df_poi_2 sans ActivityProvider sont: (189, 14)


In [58]:
#export des df clean et des df créé en csv
#csv global:
df_clean.to_csv('/content/drive/MyDrive/reunion_clean.csv')

#csv poi (sans food):
df_poi.to_csv('/content/drive/MyDrive/reunion_poi.csv')

#csv poi 2 (sans food, store and df outdoor (air activity, nautical centre, natural heritage)):
df_poi_2.to_csv('/content/drive/MyDrive/reunion_poi_2.csv')

#csv poi 3 (sans food sans activity provider sans best):
df_poi_3.to_csv('/content/drive/MyDrive/reunion_poi_3.csv')

#csv best :
df_best.to_csv('/content/drive/MyDrive/reunion_best.csv')

#csv des restaurants:
df_food.to_csv('/content/drive/MyDrive/reunion_food.csv')

#csv des activity provider:
#df_activity.to_csv('/content/drive/MyDrive/reunion_activity.csv')
df_outdooractivity.to_csv('/content/drive/MyDrive/reunion_activity.csv')

#csv ne comportant que les coordonnées des poi:
df_coordonnees.to_csv('/content/drive/MyDrive/reunion_coordonnees.csv') 

#csv de correspondance entre les villes de la réunion et leur code postal
df_ville_cp_index.to_csv('/content/drive/MyDrive/reunion_ville_cp.csv') #à importer sur sql/neo4j avec un nouvel id/clé primaire
